In [4]:
##%overwritefile
##%file:src/dart_generate_dartfile.py
##%noruncode
    def generate_dartfile(self, source_filename, binary_filename, cflags=None, ldflags=None):

        return


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/dart_generate_dartfile.py created successfully


In [5]:
##%overwritefile
##%file:src/dart_do_execute.py
##%noruncode
    def do_execute(self, code, silent, store_history=True,
                   user_expressions=None, allow_stdin=True):
        
        magics, code = self._filter_magics(code)
        if len(magics['replcmdmode'])>0:
            return self.send_replcmd(code, silent, store_history=True,
                   user_expressions=None, allow_stdin=False)
        if len(magics['noruncode'])>0 and ( len(magics['command'])>0 or len(magics['dartcmd'])>0):
            return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
        if len(magics['file'])<1:
            magics, code = self._add_main(magics, code)
        
        with self.new_temp_file(suffix='.dart',dir=os.path.abspath('')) as source_file:
            source_file.write(code)
            source_file.flush()
            newsrcfilename=source_file.name
            
            if len(magics['file'])>0:
                newsrcfilename = magics['file']
                newsrcfilename = os.path.join(os.path.abspath(''),newsrcfilename)
                if os.path.exists(newsrcfilename):
                    newsrcfilename +=".new"
                if not os.path.exists(os.path.dirname(newsrcfilename)) :
                    os.makedirs(os.path.dirname(newsrcfilename))
                os.rename(source_file.name,newsrcfilename)
                self._write_to_stdout("[Dart kernel] Info:file "+ newsrcfilename +" created successfully\n")
        if len(magics['noruncode'])>0:
            return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
        
        p = self.create_jupyter_subprocess(['dart','--verbose',newsrcfilename]+ magics['args'],cwd=None,shell=False,env=magics['env'])
        #p = self.create_jupyter_subprocess([binary_file.name]+ magics['args'],cwd=None,shell=False)
        #p = self.create_jupyter_subprocess([self.master_path, binary_file.name] + magics['args'],cwd='/tmp',shell=True)
        self.g_rtsps[str(p.pid)]=p
        if magics!=None and len(magics['showpid'])>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        while p.poll() is None:
            p.write_contents(magics)

        # wait for threads to finish, so output is always shown
        p._stdout_thread.join()
        p._stderr_thread.join()
        del self.g_rtsps[str(p.pid)]
        p.write_contents(magics)

        self.cleanup_files()
        if p.returncode != 0:
            self._write_to_stderr("[Dart kernel] Executable exited with code {}".format(p.returncode))
        return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/dart_do_execute.py created successfully


In [6]:
##%overwritefile
##%file:src/dart_filter_magics.py
##%noruncode
    def _filter_magics(self, code):

        magics = {'cflags': [],
                  'ldflags': [],
                  'file': [],
                  'include': [],
                  'templatefile': [],

                  'repllistpid': [],
                  'replcmdmode': [],
                  'replprompt': [],
                  'replsetip': "\r\n",
                  'replchildpid':"0",

                  'showpid': [],
                  'norunnotecmd': [],
                  'noruncode': [],
                  'command': [],
                  'fluttercmd': [],
                  'dartcmd': [],
                  'outputtype':'text/plain',
                  'env':None,
                  'runmode': [],
                  'pid': [],
                  'pidcmd': [],
                  'args': []}

        actualCode = ''

        for line in code.splitlines():
            orgline=line
            if line.strip().startswith('//%'):
                if line.strip()[3:] == "noruncode":
                    magics['noruncode'] += ['true']
                    continue
                elif line.strip()[3:] == "showpid":
                    magics['showpid'] += ['true']
                    continue
                elif line.strip()[3:] == "repllistpid":
                    magics['repllistpid'] += ['true']
                    self.repl_listpid()
                    continue
                elif line.strip()[3:] == "replcmdmode":
                    magics['replcmdmode'] += ['replcmdmode']
                    continue
                elif line.strip()[3:] == "replprompt":
                    magics['replprompt'] += ['replprompt']
                    continue
                elif line.strip()[3:] == "onlyrunnotecmd":
                    magics['onlyrunnotecmd'] += ['true']
                    continue
                findObj= re.search( r':(.*)',line)
                if not findObj or len(findObj.group(0))<2:
                    continue
                key, value = line.strip()[3:].split(":", 2)
                key = key.strip().lower()

                if key in ['ldflags', 'cflags']:
                    for flag in value.split():
                        magics[key] += [flag]
                elif key == "runmode":
                    if len(value)>0:
                        magics[key] = value[re.search(r'[^/]',value).start():]
                    else:
                        magics[key] ='real'
                elif key == "file":
                    if len(value)>0:
                        magics[key] = value[re.search(r'[^/]',value).start():]
                    else:
                        magics[key] ='newfile'
                elif key == "include":
                    if len(value)>0:
                        magics[key] = value
                    else:
                        magics[key] =''
                        continue
                    if len(magics['include'])>0:
                        index1=line.find('//%')
                        line=self.readcodefile(magics['include'],index1)
                        actualCode += line + '\n'
                elif key == "templatefile":
                    index1=line.find('//%')
                    if len(value)>0:
                        magics[key] =value.split(" ",1)
                    else:
                        magics[key] =None
                        continue
                    templatefile=magics['templatefile'][0]
                    if len(magics['templatefile'])>1:
                        argsstr=magics['templatefile'][1]
                        templateargsdict=_filter_dict(argsstr)
                    else:
                        templateargsdict=None
                    if len(magics['templatefile'])>0:
                        line=self.readtemplatefile(templatefile,index1,templateargsdict)
                        actualCode += line + '\n'
                elif key == "pidcmd":
                    magics['pidcmd'] = [value]
                    if len(magics['pidcmd'])>0:
                        findObj= value.split(",",1)
                        if findObj and len(findObj)>1:
                            pid=findObj[0]
                            cmd=findObj[1]
                            self.send_cmd(pid=pid,cmd=cmd)
                elif key == "replsetip":
                    magics['replsetip'] = value
                elif key == "replchildpid":
                    magics['replchildpid'] = value
                elif key == "command" or key == "cmd":
                    magics['command'] = [value]
                    if len(magics['command'])>0:
                        self.do_shell_command(magics['command'],env=magics['env'])
                elif key == "dartcmd":
                    for flag in value.split():
                        magics[key] += [flag]
                    if len(magics['dartcmd'])>0:
                        self.do_dart_command(magics['dartcmd'],magics=magics)
                elif key == "fluttercmd":
                    for flag in value.split():
                        magics[key] += [flag]
                    if len(magics['fluttercmd'])>0:
                        self.do_flutter_command(magics['fluttercmd'],magics=magics)
                elif key == "env":
                    envdict=self._filter_env(value)
                    magics[key] =dict(envdict)
                elif key == "outputtype":
                    magics[key]=value
                elif key == "args":
                    # Split arguments respecting quotes
                    for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', value):
                        magics['args'] += [argument.strip('"')]

                # always add empty line, so line numbers don't change
                # actualCode += '\n'

            # keep lines which did not contain magics
            else:
                actualCode += line + '\n'
        newactualCode=actualCode
        if len(magics['file'])>0 and len(magics['noruncode'])>0:
            newactualCode=''
            for line in actualCode.splitlines():
                line=cleantestcode(line)
                newactualCode += line
        return magics, actualCode


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/dart_filter_magics.py created successfully


In [7]:
##%overwritefile
##%file:src/dart_do_dart_command.py
##%noruncode
    def do_dart_command(self,commands=None,cwd=None,magics=None):
        p = self.create_jupyter_subprocess(['dart']+commands,cwd=os.path.abspath(''),shell=False)
        self.g_rtsps[str(p.pid)]=p
        if magics!=None and len(magics['showpid'])>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        while p.poll() is None:
            p.write_contents()
        # wait for threads to finish, so output is always shown
        p._stdout_thread.join()
        p._stderr_thread.join()
        del self.g_rtsps[str(p.pid)]
        p.write_contents()

        if p.returncode != 0:
            self._write_to_stderr("[Dart kernel] Executable exited with code {}".format(p.returncode))
        else:
            self._write_to_stdout("[Dart kernel] Info:dart command success.")
        return


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/dart_do_dart_command.py created successfully


In [8]:
##%overwritefile
##%file:src/dart_do_flutter_command.py
##%noruncode
    def do_flutter_command(self,commands=None,cwd=None,magics=None):
        p = self.create_jupyter_subprocess(['flutter']+commands,cwd=os.path.abspath(''),shell=False)
        self.g_rtsps[str(p.pid)]=p
        if magics!=None and len(magics['showpid'])>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        while p.poll() is None:
            p.write_contents()
        # wait for threads to finish, so output is always shown
        p._stdout_thread.join()
        p._stderr_thread.join()
        del self.g_rtsps[str(p.pid)]
        p.write_contents()

        if p.returncode != 0:
            self._write_to_stderr("[Dart kernel] Executable exited with code {}".format(p.returncode))
        else:
            self._write_to_stdout("[Dart kernel] Info:flutter command success.")
        return


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/dart_do_flutter_command.py created successfully


In [4]:
##%overwritefile
##%file:../jupyter-MyDart-kernel/jupyter_MyDart_kernel/kernel.py
###%file:dart_kernel.py
##%noruncode
#
#   MyDart Jupyter Kernel
#   generated by MyPython
#
##%include:src/head.py

##%include:src/common.py

##%include:src/IREPLWrapper.py
##%include:src/RealTimeSubprocess.py

class DartKernel(Kernel):
    implementation = 'jupyter_MyDart_kernel'
    implementation_version = '1.0'
    language = 'Dart'
    language_version = '2.X.X'
    language_info = {'name': 'text/x-csrc',
                     'mimetype': 'text/x-csrc',
                     'file_extension': '.dart'}
    banner = "Dart kernel.\n" \
             "Uses Dart, compiles in dart, and creates source code files and executables in temporary folder.\n"

    main_head = "\n" \
            "\n" \
            "int main(List<String> arguments){\n"

    main_foot = "\nreturn 0;\n}"
    
##%include:src/comm_attribute.py

    def __init__(self, *args, **kwargs):
        super(DartKernel, self).__init__(*args, **kwargs)
        self.kernelinfo='[MyDart Kernel]'
        self._allow_stdin = True
        self.readOnlyFileSystem = False
        self.bufferedOutput = True
        self.linkMaths = True # always link math library
        self.wAll = True # show all warnings by default
        self.wError = False # but keep comipiling for warnings
        self.files = []
        self.resDir = path.join(path.dirname(path.realpath(__file__)), 'resources')

        self.chk_replexit_thread = Thread(target=self.chk_replexit, args=(self.g_rtsps))
        self.chk_replexit_thread.daemon = True
        self.chk_replexit_thread.start()

##%include:src/_templateHander.py
##%include:src/_readtemplatefile.py

##%include:src/cleannotes.py
##%include:src/kernel_common.py
##%include:src/_is_specialID.py

##%include:src/repl_listpid.py
##%include:src/chk_replexit.py
##%include:src/cleanup_files.py
##%include:src/new_temp_file.py
##%include:src/log_output.py
##%include:src/_write_display_data.py
##%include:src/_write_to_stdout.py
##%include:src/_write_to_stderr.py
##%include:src/_read_from_stdin.py
##%include:src/readcodefile.py
    #####################################################################
##%include:src/_start_replprg.py
##%include:src/process_output.py
##%include:src/send_replcmd.py
    #####################################################################
##%include:src/do_shell_command.py
    
##%include:src/dart_do_dart_command.py
    
##%include:src/dart_do_flutter_command.py
    
##%include:src/send_cmd.py
##%include:src/create_jupyter_subprocess.py
##%include:src/dart_generate_dartfile.py
##%include:src/compile_with_Python2native.py
##%include:src/_filter_env.py
#####################################################################
##%include:src/dart_filter_magics.py
#####################################################################
##%include:src/_add_main.py
##%include:src/dart_do_execute.py
##%include:src/do_shutdown.py


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/../jupyter-MyDart-kernel/jupyter_MyDart_kernel/kernel.py created successfully
